In [ ]:
## a useful function to generate a data list for further analysis
import os,json,sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import linalg as LA
from scipy import stats
import scipy.stats as st
import h5py
from sorting_data_pairwise_comparison import calculate_relative_position
##need to add this additional cell because useful tools are in another folder. Need to integrate these two folders one day
current_working_directory = Path.cwd()
parent_dir = current_working_directory.resolve().parents[0]
sys.path.insert(0, str(parent_dir) + "\\utilities")
from useful_tools import select_animals_gpt,find_file
from data_cleaning import findLongestConseqSubseq

In [ ]:
json_file = "./analysis_methods_dictionary.json"
with open(json_file, "r") as f:
    analysis_methods = json.loads(f.read())
    
#Put the folder of your Unity folder below
#thisDataset ="D:/MatrexVR_Swarm_Data/RunData"
#thisDataset ="D:/MatrexVR_blackbackground_Data/RunData"
#thisDataset ="D:/MatrexVR_grass1_Data/RunData"
thisDataset ="D:/MatrexVR_2024_Data/RunData"
#parameter name means independent variable in the experiment
#variable_name='mu'
variable_name='location'
#variable_name='agent_speed'
#check trace in trial 115 from VR1_2024-11-16_155242_score_full, maybe there is a jump
analysis_window=analysis_methods.get("analysis_window")
monitor_fps=analysis_methods.get("monitor_fps")

#### Session 1.0: select animals based on condition and return which a directory list and a list of vr rig number to specify which animal to analyse

In [ ]:
# Define the path to your Excel file
dir_list = []
file_type=".h5"
using_google_sheet=True
sheet_name = "Unity_MatrexVR"
experiment_name=analysis_methods.get("experiment_name")
# if type(thisDataset) == str:
#     thisDataset = Path(thisDataset)
if analysis_methods.get("load_individual_data") == True:
    if using_google_sheet==True:
        # database_id = "1UL4eEUrQMapx9xz11-IyOSlPBcep3I9vBJ2uGgVudb8"
        #         #https://docs.google.com/spreadsheets/d/1UL4eEUrQMapx9xz11-IyOSlPBcep3I9vBJ2uGgVudb8/edit?usp=sharing
        # url = f"https://docs.google.com/spreadsheets/d/{database_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        database_id = "1UL4eEUrQMapx9xz11-IyOSlPBcep1I9vBJ2uGgVudb8"
                #https://docs.google.com/spreadsheets/d/1UL4eEUrQMapx9xz11-IyOSlPBcep1I9vBJ2uGgVudb8/edit?usp=sharing
        url = f"https://docs.google.com/spreadsheets/d/{database_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
        #df = pd.read_excel(url, engine='openpyxl')## use this function if the file is not google sheet but uploaded excel file
        df = pd.read_csv(url)
    else:
        excel_file_path = "Z:/DATA/experiment_trackball_Optomotor/Locusts Management.xlsx"
        print(f"using a database {excel_file_path} from the server but this file might be outdated")
        # Create a 'with' statement to open and read the Excel file
        with pd.ExcelFile(excel_file_path) as xls:
            # Read the Excel sheet into a DataFrame with the sheet name (folder name)
            df = pd.read_excel(xls, sheet_name)
        ##list up the conditions and answers as strings for input argument to select animal. One condition must pair with one answer
    if analysis_methods.get("select_animals_by_condition") == True:
       #animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)","gregarious_leader_grass","Excluding this animal from analysis (Usually when animals die or molt, T/F)","F")
        #animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)","gregarious_leader_black","Excluding this animal from analysis (Usually when animals die or molt, T/F)","F")
        #animal_of_interest=select_animals_gpt(df,"Independent variable (list up all of them in the experiment)",variable_name)
        animal_of_interest=select_animals_gpt(df,"Independent variable1",variable_name,"Independent variable2","bifuration_vr_locust_sta_black_locust","Excluding this animal from analysis (Usually when animals die or molt, T/F)","F")
        #animal_of_interest=select_animals_gpt(df,"Independent variable1",variable_name,"Excluding this animal from analysis (Usually when animals die or molt, T/F)","F")
    else:
        animal_of_interest=df
    folder_name=animal_of_interest["folder name"].values

    dir_tile=np.tile(thisDataset, (len(folder_name), 1))
    vr_no=animal_of_interest["VR number"].values
    dir_list = [''.join([x[0], '/', y]) for x,y in zip(dir_tile,folder_name)]
    #dir_dict = itertools.zip_longest(dir_list, vr_no.tolist())
    dir_dict = zip(dir_list, vr_no.tolist())
else:
    for root, dirs, files in os.walk(thisDataset):
        for folder in dirs:
            folder_path=os.path.join(root,folder)
            if any(name.endswith(file_type) for name in os.listdir(folder_path)):
                dir_list.append(folder_path.replace("\\", "/"))

### Session 1.1: introduce helper functions to make plot and calculate speed

In [ ]:
analysis_methods.update({"plotting_trajectory": False})

In [ ]:
def plot_relative_pos_distribution(relative_pos_of_interest,trial_type_of_interest,distance_threshold_for_plotting,analysis_methods,this_vr='all'):
    save_output= analysis_methods.get("save_output")
    xlimit=(0,distance_threshold_for_plotting)
    ylimit=(-1*distance_threshold_for_plotting,distance_threshold_for_plotting)
    fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2,figsize=(18, 6),tight_layout=True,sharex=True, sharey=True)
    if len(relative_pos_of_interest)==1:
        ax1.hist2d(relative_pos_of_interest[0]['x'].values,relative_pos_of_interest[0]['y'].values,bins=50)
    else:
        ax1.hist2d(np.hstack((relative_pos_of_interest[0]['x'].values,relative_pos_of_interest[1]['x'].values)),np.hstack((relative_pos_of_interest[0]['y'].values,relative_pos_of_interest[1]['y'].values)),bins=50)
        ax2.hist2d(np.hstack((relative_pos_of_interest[0]['x'].values,relative_pos_of_interest[1]['x'].values)),np.hstack((relative_pos_of_interest[0]['y'].values,relative_pos_of_interest[1]['y'].values*-1)),bins=50)
        ax2.set(yticks=[-1*distance_threshold_for_plotting,-5,0,5,distance_threshold_for_plotting],xticks=[0,5,distance_threshold_for_plotting],xlim=xlimit,ylim=ylimit,title='agent preference',adjustable='box', aspect='equal')
    ax1.set(
    yticks=[-1*distance_threshold_for_plotting,-5,0,5,distance_threshold_for_plotting],
    xticks=[0,5,distance_threshold_for_plotting],
    xlim=xlimit,ylim=ylimit,title='spatial preference',adjustable='box', aspect='equal')
    if len(relative_pos_of_interest)==1:
        fig_name=f"heatmap2D_{this_vr}_{trial_type_of_interest[0]}_distance_threshold_{distance_threshold_for_plotting}.png"
    else:
        fig_name=f"heatmap2D_{this_vr}_{trial_type_of_interest[0]}_and_{trial_type_of_interest[1]}_distance_threshold_{distance_threshold_for_plotting}.png"
    if save_output==True:
        fig.savefig(fig_name)
    plt.show()

In [ ]:
dir_list=dir_list[-26:]
vr_no=vr_no[-26:]

In [ ]:
relative_pos_all_animals=[]
raster_across_animals=[]
trial_evaluation_across_animals=[]
animal_id=0
for this_dir,this_vr in zip(dir_list,vr_no):
    if Path(this_dir).is_dir()==False:
        continue
    agent_pattern = f"VR{this_vr}*agent_full.h5"
    xy_pattern = f"VR{this_vr}*XY_full.h5"
    summary_pattern = f"VR{this_vr}*score_full.h5"
    agent_file = find_file(Path(this_dir), agent_pattern)
    focal_animal_file = find_file(Path(this_dir), xy_pattern)
    summary_file = find_file(Path(this_dir), summary_pattern)
    relative_pos,trial_evaluation_list,raster_pd,num_unfilled_gap,simulated_across_trials_pd=calculate_relative_position(summary_file,focal_animal_file,agent_file,analysis_methods)
    if animal_id==0:
        largest_unfilled_gap=num_unfilled_gap
    elif num_unfilled_gap>largest_unfilled_gap:
        largest_unfilled_gap=num_unfilled_gap
    else:
        pass
    relative_pos_all_animals.append(relative_pos)
    trial_evaluation=pd.concat(trial_evaluation_list)
    trial_evaluation.insert(0, 'animal_id',np.repeat(animal_id,trial_evaluation.shape[0]))
    trial_evaluation_across_animals.append(trial_evaluation)
    raster_pd.insert(0, 'animal_id', np.repeat(animal_id,raster_pd.shape[0]))
    raster_across_animals.append(raster_pd)
    animal_id=animal_id+1
analysis_methods['largest_unfilled_gap']= largest_unfilled_gap

In [ ]:
## calculating the relative distance between the focal animal and other agents in every frame
relative_pos_all_pd=pd.concat(relative_pos_all_animals,ignore_index=True)
relative_pos_all_pd['distance']=relative_pos_all_pd.apply(lambda row:np.sqrt(np.square(row['x']) + np.square(row['y'])), axis=1)

In [ ]:
## get trial type and sort them by the length of the name. In this case, homogeneous trial should usually be the first twos
trial_type_list=sorted(relative_pos_all_pd['type'].unique(), key=len)
trial_type_of_interest=[trial_type_list[0]]
trial_type_of_interest=[trial_type_list[1]]
trial_type_of_interest=[trial_type_list[2]]
trial_type_of_interest=[trial_type_list[3]]
trial_type_of_interest=[trial_type_list[0],trial_type_list[1]]
trial_type_of_interest=[trial_type_list[2],trial_type_list[3]]

In [ ]:
analysis_methods.update({"save_output":True})
trial_type_of_interest=[trial_type_list[2],trial_type_list[3]]
for distance_threshold_for_plotting in range(10,30,10):
    print(f"distance threshold for plotting 2D histogram: {distance_threshold_for_plotting} cm")
    if len(trial_type_of_interest)==1:
        relative_pos_of_interest=[relative_pos_all_pd[(relative_pos_all_pd['type']==trial_type_of_interest[0]) & (relative_pos_all_pd['distance']<distance_threshold_for_plotting)]]
    elif len(trial_type_of_interest)==2:
        relative_pos_of_interest=[relative_pos_all_pd[(relative_pos_all_pd['type']==trial_type_of_interest[0]) & (relative_pos_all_pd['distance']<distance_threshold_for_plotting)],relative_pos_all_pd[(relative_pos_all_pd['type']==trial_type_of_interest[1]) & (relative_pos_all_pd['distance']<distance_threshold_for_plotting)]]
    plot_relative_pos_distribution(relative_pos_of_interest,trial_type_of_interest,distance_threshold_for_plotting,analysis_methods)

In [ ]:
def calculate_preference_index(relative_pos_all_animals,trial_type_of_interest,thresholds=[4,5,6,7,8]):
    spatial_preference_animals=np.zeros((len(thresholds),len(relative_pos_all_animals),relative_pos_all_animals[0]['type'].unique().shape[0]))
    social_preference_animals=np.zeros((len(thresholds),len(relative_pos_all_animals),relative_pos_all_animals[0]['type'].unique().shape[0]))
    epochs_l_animals=np.zeros((len(thresholds),len(relative_pos_all_animals),relative_pos_all_animals[0]['type'].unique().shape[0]))
    epochs_r_animals=np.zeros((len(thresholds),len(relative_pos_all_animals),relative_pos_all_animals[0]['type'].unique().shape[0]))
    epochs_exp_animals=np.zeros((len(thresholds),len(relative_pos_all_animals),relative_pos_all_animals[0]['type'].unique().shape[0]))
    epochs_con_animals=np.zeros((len(thresholds),len(relative_pos_all_animals),relative_pos_all_animals[0]['type'].unique().shape[0]))
    for i in range(len(relative_pos_all_animals)):
        this_animal_relation=relative_pos_all_animals[i]
        type_no=0
        # epochs_exp_list=[]
        # epochs_con_list=[]
        # epochs_r_list=[]
        # epochs_l_list=[]
        # relative_pos_list1=[]
        # relative_pos_list2=[] 
        # relative_distance_list1=[]
        # relative_distance_list2=[]
        for key,grp in this_animal_relation.groupby('type'):
            print(key)
            focal_xy = np.vstack((grp["x"].values,grp["y"].values))
            relative_distance=LA.norm(focal_xy,axis=0)
            # if key==trial_type_of_interest[1]:
            #     relative_pos_list1.append(focal_xy)
            #     relative_distance_list1.append(relative_distance)
            # elif key==trial_type_of_interest[0]:
            #     relative_pos_list2.append(focal_xy)
            #     relative_distance_list2.append(relative_distance)        
            for j,this_threshold in enumerate(thresholds):
                epochs_forR=np.sum(grp[relative_distance<this_threshold]["agent_id"].values)
                epochs_forL=grp[relative_distance<this_threshold]["agent_id"].values.shape[0]-np.sum(grp[relative_distance<this_threshold]["agent_id"].values)
                preferL_index=(epochs_forL-epochs_forR)/(epochs_forR+epochs_forL)
                spatial_preference_animals[j,i,type_no]=preferL_index
                if key==trial_type_of_interest[1]:
                    print(key,type_no)
                    epochs_exp=epochs_forR
                    epochs_con=epochs_forL
                    epochs_l_animals[j,i,type_no]=epochs_forL
                    epochs_r_animals[j,i,type_no]=epochs_forR
                    epochs_exp_animals[j,i,type_no]=epochs_exp
                    epochs_con_animals[j,i,type_no]=epochs_con
                elif key==trial_type_of_interest[0]:
                    print(key,type_no)
                    epochs_exp=epochs_forL
                    epochs_con=epochs_forR
                    social_preference_animals[j,i,type_no]=prefer_exp_index
                    epochs_l_animals[j,i,type_no]=epochs_forL
                    epochs_r_animals[j,i,type_no]=epochs_forR
                    epochs_exp_animals[j,i,type_no]=epochs_exp
                    epochs_con_animals[j,i,type_no]=epochs_con
                else:
                    epochs_exp=np.nan
                    epochs_con=np.nan
                prefer_exp_index=(epochs_exp-epochs_con)/(epochs_exp+epochs_con)
                social_preference_animals[j,i,type_no]=prefer_exp_index
            type_no=type_no+1
    if trial_type_of_interest[0]=='LeaderLocust' or trial_type_of_interest[1]=='LeaderLocust':
        row_of_interest=0
    else:
        row_of_interest=1
    left_right_preference_across_animals=(epochs_l_animals[:,:,row_of_interest]+epochs_l_animals[:,:,row_of_interest+2]-epochs_r_animals[:,:,row_of_interest]-epochs_r_animals[:,:,row_of_interest+2])/(epochs_l_animals[:,:,row_of_interest]+epochs_l_animals[:,:,row_of_interest+2]+epochs_r_animals[:,:,row_of_interest]+epochs_r_animals[:,:,row_of_interest+2])
    exp_con_preference_across_animals=(epochs_exp_animals[:,:,row_of_interest]+epochs_exp_animals[:,:,row_of_interest+2]-epochs_con_animals[:,:,row_of_interest]-epochs_con_animals[:,:,row_of_interest+2])/(epochs_exp_animals[:,:,row_of_interest]+epochs_exp_animals[:,:,row_of_interest+2]+epochs_con_animals[:,:,row_of_interest]+epochs_con_animals[:,:,row_of_interest+2])
    return left_right_preference_across_animals,exp_con_preference_across_animals

In [ ]:
trial_type_of_interest=[trial_type_list[2],trial_type_list[3]]

In [ ]:
trial_type_of_interest

In [ ]:
thresholds=[4,5,6,7,8]
left_right_preference_across_animals,exp_con_preference_across_animals=calculate_preference_index(relative_pos_all_animals,trial_type_of_interest,thresholds=thresholds)

In [ ]:
def plot_preference_index(left_right_preference_across_animals,exp_con_preference_across_animals,trial_type_of_interest,thresholds,analysis_methods,this_vr='all'):
    save_output= analysis_methods.get("save_output")
    object_of_interest=trial_type_of_interest[0].split("_x_")
    if len(trial_type_of_interest)==1:
        fig_name=f"preference_index_{this_vr}_condition_{trial_type_of_interest[0]}"
        data_color='k'
    elif len(trial_type_of_interest[0].split("_x_"))==1:
        fig_name=f"preference_index_{this_vr}_homegenous_condition_{trial_type_of_interest[0]}_and_{trial_type_of_interest[1]}"
        data_color='k'
    else:
        fig_name=f"preference_index_{this_vr}_top_{object_of_interest[1]}_bottom_{object_of_interest[0]}"
        data_color='r'
    preference_plot, (ax1,ax2) = plt.subplots(
        nrows=1, ncols=2, figsize=(20, 4), tight_layout=True
    )
    xaxis_points=np.repeat(thresholds,len(relative_pos_all_animals)).reshape(len(thresholds),len(relative_pos_all_animals))
    ax1.scatter(xaxis_points,left_right_preference_across_animals,alpha=0.2,c=data_color)
    mean_index=np.mean(xaxis_points,axis=1)
    mean_response=np.nanmean(left_right_preference_across_animals,axis=1)
    sem_response=np.nanstd(left_right_preference_across_animals, axis=1, ddof=1) / np.sqrt(left_right_preference_across_animals.shape[1])
    ax1.errorbar(
        mean_index,
        mean_response,
        yerr=sem_response,
        c=data_color,
        fmt="o",
        elinewidth=2,
        capsize=3,
    )
    ax1.set(
        ylabel="(postive means prefer left)",
        yticks=[-1,0,1],
        xticks=thresholds,
        xlabel="ROI size (cm)",
        title="spatial preference"
    )
    ax2.scatter(xaxis_points,exp_con_preference_across_animals,alpha=0.2,c=data_color)
    mean_index=np.mean(xaxis_points,axis=1)
    mean_response=np.nanmean(exp_con_preference_across_animals,axis=1)
    sem_response=np.nanstd(exp_con_preference_across_animals, axis=1, ddof=1) / np.sqrt(exp_con_preference_across_animals.shape[1])
    ax2.errorbar(
        mean_index,
        mean_response,
        yerr=sem_response,
        c=data_color,
        fmt="o",
        elinewidth=2,
        capsize=3,
    )
    ax2.set(
        ylabel="(positive means prefer exp)",
        yticks=[-1,0,1],
        xticks=thresholds,
        xlabel="ROI size (cm)",
        title="visual preference"
    )
    if save_output==True:
        preference_plot.savefig(f"{fig_name}.png")
        preference_plot.savefig(f"{fig_name}.svg")

In [ ]:
plot_preference_index(left_right_preference_across_animals,exp_con_preference_across_animals,trial_type_of_interest,thresholds,analysis_methods)

In [ ]:
#preference_threshold_animals format (thresholds x animals x type)

left_right_preference, subplots = plt.subplots(
    nrows=1, ncols=spatial_preference_animals.shape[2], figsize=(20, 4), tight_layout=True
)
xaxis_points=np.repeat(thresholds,len(relative_pos_all_animals)).reshape(len(thresholds),len(relative_pos_all_animals))
for choice_type in range(spatial_preference_animals.shape[2]):
    subplots[choice_type].scatter(xaxis_points,spatial_preference_animals[:,:,choice_type],alpha=0.2)
    subplots[choice_type].set_xlabel("ROI size (cm)")
    mean_index=np.mean(xaxis_points,axis=1)
    mean_response=np.nanmean(spatial_preference_animals[:,:,choice_type],axis=1)
    sem_response=np.nanstd(spatial_preference_animals[:,:,choice_type], axis=1, ddof=1) / np.sqrt(
    spatial_preference_animals[:,:,choice_type].shape[1]
)
    subplots[choice_type].errorbar(
        mean_index,
        mean_response,
        yerr=sem_response,
        c='k',
        fmt="o",
        elinewidth=2,
        capsize=3,
    )
    #
subplots[0].set_ylabel("preference index (postive means prefer left)")